In [1]:
import pandas as pd
import tweepy

# Leer las llaves
keys = pd.read_csv("mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [2]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [18]:
tweets = tweepy.Cursor(api.search,
                       q=['atletico nacional -RT'],
                       lang='es',
                       geocode='3.42158,-76.5205,1000km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [19]:
t = [tweet for tweet in tweets]

In [20]:
t[0].created_at

datetime.datetime(2021, 5, 20, 18, 41, 37)

In [21]:
t[0].user.location

'Medellín, Colombia'

In [23]:
t[50].full_text

'🏆 COPA CONMEBOL LIBERTADORES\n🗓️ Jueves 20/05/2021\n\n⌚19:00 - Argentinos vs Atlético Nacional\n⌚21:00 - Boca vs Barcelona\n⌚21:00 - Olimpia vs Internacional\n\n#Libertadores #CopaLibertadores'

In [24]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Medellín')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,"Ahora bien, según los jugadores que convocó el...",2021-05-20 18:41:37,Medellín,"Medellín, Colombia",1395449652376489984,https://twitter.com/i/web/stuatus/139544965237...
1,EN VIVO: Argentinos Juniors Vs. Atlético Nacio...,2021-05-20 18:21:12,Medellín,"Bogotá, Colombia",1395444512554508288,https://twitter.com/i/web/stuatus/139544451255...
2,"@Alfredo87340960 Harían tiene buenos números, ...",2021-05-20 18:07:52,Medellín,"Valledupar, Colombia",1395441155156889601,https://twitter.com/i/web/stuatus/139544115515...
3,Copa Libertadores | Argentinos Juniors vs. Atl...,2021-05-20 18:04:46,Medellín,Colombia,1395440377319104516,https://twitter.com/i/web/stuatus/139544037731...
4,"Atlético Nacional, análisis del rival: Argenti...",2021-05-20 17:27:17,Medellín,Colombia,1395430941737328640,https://twitter.com/i/web/stuatus/139543094173...


In [25]:
data.to_csv('archivotwitter.csv', index=False)

In [27]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,"Ahora bien, según los jugadores que convocó el...",2021-05-20 18:41:37,Medellín,"Medellín, Colombia",1395449652376489984,https://twitter.com/i/web/stuatus/139544965237...,"Ahora bien, según los jugadores que convocó el...",[],[#MiOnceEnVavelNacional],[]
1,EN VIVO: Argentinos Juniors Vs. Atlético Nacio...,2021-05-20 18:21:12,Medellín,"Bogotá, Colombia",1395444512554508288,https://twitter.com/i/web/stuatus/139544451255...,EN VIVO: Argentinos Juniors Vs. Atlético Nacio...,"[https://t.co/5KT3d8OXTx, https://t.co/lWSR8L9...",[],[]
2,"@Alfredo87340960 Harían tiene buenos números, ...",2021-05-20 18:07:52,Medellín,"Valledupar, Colombia",1395441155156889601,https://twitter.com/i/web/stuatus/139544115515...,"Harían tiene buenos números, es el 3er mejor ...",[],[],[@Alfredo87340960]
3,Copa Libertadores | Argentinos Juniors vs. Atl...,2021-05-20 18:04:46,Medellín,Colombia,1395440377319104516,https://twitter.com/i/web/stuatus/139544037731...,Copa Libertadores | Argentinos Juniors vs. Atl...,[https://t.co/9gMZkjnRbT],[],[]
4,"Atlético Nacional, análisis del rival: Argenti...",2021-05-20 17:27:17,Medellín,Colombia,1395430941737328640,https://twitter.com/i/web/stuatus/139543094173...,"Atlético Nacional, análisis del rival: Argenti...","[https://t.co/wToXMFMcLu, https://t.co/7D4BHrj...",[],[]
...,...,...,...,...,...,...,...,...,...,...
95,¿ATLÉTICO NACIONAL ES LA BASE DE LA SELECCIÓN ...,2021-05-20 01:14:27,Medellín,"Medellín, Antioquia",1395186121584398339,https://twitter.com/i/web/stuatus/139518612158...,¿ATLÉTICO NACIONAL ES LA BASE DE LA SELECCIÓN ...,[https://t.co/QzlrbgOdHz],[#10SportsCo],[]
96,@___valeenn @IbaiLlanos Pero si hoy x hoy atle...,2021-05-20 01:13:55,Medellín,"Bogotá, D.C., Colombia",1395185986548772865,https://twitter.com/i/web/stuatus/139518598654...,Pero si hoy x hoy atletico nacional y junior...,[],[],"[@___valeenn, @IbaiLlanos]"
97,"@___valeenn @IbaiLlanos Atlético Nacional, Jun...",2021-05-20 01:13:17,Medellín,"Bogotá, D.C., Colombia",1395185827186192391,https://twitter.com/i/web/stuatus/139518582718...,"Atlético Nacional, Junior De Barranquilla, A...",[],[],"[@___valeenn, @IbaiLlanos]"
98,DE LA MANO DE RUEDA Y LOS VERDOLAGAS RUMBO A Q...,2021-05-20 01:11:34,Medellín,Medellín,1395185397102166016,https://twitter.com/i/web/stuatus/139518539710...,DE LA MANO DE RUEDA Y LOS VERDOLAGAS RUMBO A Q...,[https://t.co/vVZejbgS7d],[],[]


In [ ]:
PUNTO 2 

In [28]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [30]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions,emojis
0,"Ahora bien, según los jugadores que convocó el...",2021-05-20 18:41:37,Medellín,"Medellín, Colombia",1395449652376489984,https://twitter.com/i/web/stuatus/139544965237...,"Ahora bien, según los jugadores que convocó el...",[],[#MiOnceEnVavelNacional],[],"[🤔, 💚]"
1,EN VIVO: Argentinos Juniors Vs. Atlético Nacio...,2021-05-20 18:21:12,Medellín,"Bogotá, Colombia",1395444512554508288,https://twitter.com/i/web/stuatus/139544451255...,EN VIVO: Argentinos Juniors Vs. Atlético Nacio...,"[https://t.co/5KT3d8OXTx, https://t.co/lWSR8L9...",[],[],[]
2,"@Alfredo87340960 Harían tiene buenos números, ...",2021-05-20 18:07:52,Medellín,"Valledupar, Colombia",1395441155156889601,https://twitter.com/i/web/stuatus/139544115515...,"Harían tiene buenos números, es el 3er mejor ...",[],[],[@Alfredo87340960],[]
3,Copa Libertadores | Argentinos Juniors vs. Atl...,2021-05-20 18:04:46,Medellín,Colombia,1395440377319104516,https://twitter.com/i/web/stuatus/139544037731...,Copa Libertadores | Argentinos Juniors vs. Atl...,[https://t.co/9gMZkjnRbT],[],[],[]
4,"Atlético Nacional, análisis del rival: Argenti...",2021-05-20 17:27:17,Medellín,Colombia,1395430941737328640,https://twitter.com/i/web/stuatus/139543094173...,"Atlético Nacional, análisis del rival: Argenti...","[https://t.co/wToXMFMcLu, https://t.co/7D4BHrj...",[],[],[]


In [31]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [32]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
16,⭐,6
3,⚽,6
11,🏆,6
30,⌚,5
10,⬇️,4


In [36]:

trace = go.Bar(x=temp_emojis.emoji.values[:25],
               y=temp_emojis.cnt.values[:25])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)